# Gaussian_Mixture_Model

`TODO`: someone else should implement this. Ian took a class on this and doesn't feel 
comfortable implementing it, as it would basically be a re-implementation of what he did, and he doesn't want to hit a AIV. 

In [ ]:
import numpy as np